# 🫁 Pneumothorax AI — Global Pre-training
**TÜBİTAK 2209-A | Ahmet Demir | Dokuz Eylül Üniversitesi**

**Veri stratejisi — Google Drive:**
- NIH verileri Drive'a bir kez indirilir, sonraki oturumlarda tekrar indirilmez
- Tüm checkpoint'ler Drive'a kaydedilir
- Drive path: `MyDrive/tubitak_pneumothorax/`

**Tahmini süre:** T4 GPU ile ~12 saat (50 epoch, 112k görüntü)

## 0. GPU Kontrolü

In [ ]:
import torch
print('GPU:', torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'YOK — Runtime > GPU seç!')
print('CUDA:', torch.version.cuda)
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

## 1. Google Drive Bağla

In [ ]:
import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

# Drive klasör yapısı
DRIVE_BASE = '/content/drive/MyDrive/tubitak_pneumothorax'
DRIVE_SIIM = f'{DRIVE_BASE}/data/siim'
DRIVE_CKPT = f'{DRIVE_BASE}/checkpoints'

for d in [DRIVE_SIIM, DRIVE_CKPT]:
    os.makedirs(d, exist_ok=True)

print(f'✓ Drive bağlandı')
print(f'  SIIM data : {DRIVE_SIIM}')
print(f'  Checkpoint: {DRIVE_CKPT}')

## 2. Projeyi Klonla

In [ ]:
REPO = 'https://github.com/ahmetai-cell/pneumothorax-ai-detection'
PROJECT_DIR = '/content/pneumothorax-ai-detection'

if os.path.exists(PROJECT_DIR):
    !cd {PROJECT_DIR} && git pull
else:
    !git clone {REPO}

os.chdir(PROJECT_DIR)
print('Çalışma dizini:', os.getcwd())

## 3. Bağımlılıkları Yükle

In [ ]:
!pip install -q \
    segmentation-models-pytorch \
    albumentations \
    pydicom \
    pynrrd \
    wandb \
    tqdm \
    fpdf2 \
    plotly

print('✓ Kurulum tamamlandı')

## 4. Kaggle API Token Ayarla

In [ ]:
import json

KAGGLE_TOKEN = 'KGAT_ab6d3f32951d02e0cc877e406e207b72'
KAGGLE_USER  = 'salihekmen'

os.environ['KAGGLE_API_TOKEN'] = KAGGLE_TOKEN
os.environ['KAGGLE_USERNAME']  = KAGGLE_USER

kaggle_dir = os.path.expanduser('~/.kaggle')
os.makedirs(kaggle_dir, exist_ok=True)
with open(f'{kaggle_dir}/kaggle.json', 'w') as f:
    json.dump({'username': KAGGLE_USER, 'key': KAGGLE_TOKEN}, f)
os.chmod(f'{kaggle_dir}/kaggle.json', 0o600)

print('✓ Kaggle token ayarlandı')

## 5. SIIM-ACR Pneumothorax — Drive'dan Yükle veya İndir

In [ ]:
import subprocess

SIIM_LOCAL = 'data/raw/global/siim'
os.makedirs(SIIM_LOCAL, exist_ok=True)

# ── 1) Drive'da veri var mı? ──────────────────────────────────────────────
drive_count = int(subprocess.check_output(
    f'find {DRIVE_SIIM} -name "*.dcm" 2>/dev/null | wc -l', shell=True
).decode().strip())

if drive_count > 1000:
    print(f'✓ Drive\'da {drive_count:,} SIIM DICOM bulundu — symlink oluşturuluyor...')
    if not os.path.islink(SIIM_LOCAL):
        os.rmdir(SIIM_LOCAL)
        os.symlink(DRIVE_SIIM, SIIM_LOCAL)
    print(f'✓ {SIIM_LOCAL} → {DRIVE_SIIM}')

# ── 2) Drive'da yok → Kaggle dataset mirror'dan indir ────────────────────
else:
    print('Drive\'da SIIM bulunamadı — Kaggle dataset\'ten indiriliyor (~12 GB)...')
    print('⚠️  Bu işlem ~20-40 dakika sürebilir, bir kez yapılır.')
    !kaggle datasets download \
        -d jesperdramsch/siim-acr-pneumothorax-segmentation-data \
        -p {DRIVE_SIIM} \
        --unzip
    # Symlink
    if not os.path.islink(SIIM_LOCAL):
        os.rmdir(SIIM_LOCAL)
        os.symlink(DRIVE_SIIM, SIIM_LOCAL)
    downloaded = int(subprocess.check_output(
        f'find {DRIVE_SIIM} -name "*.dcm" | wc -l', shell=True
    ).decode().strip())
    print(f'✓ İndirme tamamlandı: {downloaded:,} DICOM — Drive\'a kaydedildi')

## 6. Manifest Oluştur

In [ ]:
!python scripts/data_manager.py --convert_rle
!python scripts/data_manager.py --build_manifest

## 7. W&B Giriş

In [ ]:
!pip install -q --upgrade wandb

import wandb
WANDB_KEY = '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'
wandb.login(key=WANDB_KEY, relogin=True)
print('✓ W&B bağlantısı tamam')

## 8. Pre-training Başlat

**T4 GPU'da tahmini süre:**
- 1 epoch (112k görüntü, batch=32): ~14 dakika
- 50 epoch: ~12 saat

> ⚠️ Colab oturumu ~12 saatte kapanabilir. Checkpoint her fold'dan sonra kaydedilir.

In [ ]:
!python scripts/train_global.py \
    --sources SIIM \
    --encoder efficientnet-b0 \
    --img_size 512 \
    --epochs 30 \
    --batch_size 16 \
    --num_folds 5 \
    --lr 1e-4 \
    --no_wandb \
    --checkpoint_dir {DRIVE_CKPT}

## 9. Checkpoint'i Drive'a Kopyala

In [ ]:
import shutil, glob

base_model = 'checkpoints/global_base_model.pth'
if os.path.exists(base_model):
    shutil.copy(base_model, f'{DRIVE_CKPT}/global_base_model.pth')
    shutil.copy('checkpoints/global_base_model_meta.json',
                f'{DRIVE_CKPT}/global_base_model_meta.json')
    print('✓ Base model Drive\'a kopyalandı')

for ckpt in glob.glob('checkpoints/global_folds/*.pth'):
    shutil.copy(ckpt, DRIVE_CKPT)
    print(f'  → {os.path.basename(ckpt)}')

results_csv = 'results/global_kfold_results.csv'
if os.path.exists(results_csv):
    shutil.copy(results_csv, DRIVE_CKPT)
    print('✓ Results CSV kopyalandı')

print(f'\n✓ Tüm dosyalar Drive\'a kaydedildi: {DRIVE_CKPT}')

## 10. Sonuçlar

In [ ]:
import pandas as pd
import json

try:
    df = pd.read_csv('results/global_kfold_results.csv')
    print('=== K-FOLD SONUÇLARI ===')
    print(df.to_string(index=False))
    print(f'\nOrtalama Dice : {df["best_dice"].mean():.4f} ± {df["best_dice"].std():.4f}')
    print(f'Ortalama AUC  : {df["best_auc"].mean():.4f} ± {df["best_auc"].std():.4f}')
except FileNotFoundError:
    print('Results CSV bulunamadı — eğitim henüz bitmemiş olabilir')

try:
    meta = json.load(open('checkpoints/global_base_model_meta.json'))
    print('\n=== BASE MODEL ===')
    for k, v in meta.items():
        print(f'  {k}: {v}')
except FileNotFoundError:
    pass

---
## ✅ Sonraki Adım — Fine-tuning

DEU DICOM + NRRD verileri geldiğinde:

```bash
# Checkpoint'i Drive'dan al
cp /content/drive/MyDrive/tubitak_pneumothorax/checkpoints/global_base_model.pth checkpoints/

# DEU verilerini koy
# data/local/dicom/*.dcm
# data/local/nrrd/*.nrrd

# Fine-tune et (~1-2 saat)
python scripts/fine_tune_local.py --freeze_encoder --epochs 20
```